### Attivazione del server

Come in ogni interazione client-server, è necessario prima di tutto che il server sia attivo e raggiungibile. MCP consente interazioni sia via standard IO sia via http. Eseguire perciò dalla linea di comando:
* `python xlserver.py stdio`  
oppure per esempio:
* `python xlserver.py http 127.0.0.1 8000`

### Connessione del client al server

In base a come si è eseguito il server, se via stdio o http, si può connettere il client al server scegliendo l'indirizzo a cui connettersi:
* `server_address = "xlserver.py"`  
oppure per esempio:
* `server_address = "http://127.0.0.1:8000/mcp"`

In [4]:
import client_tools

server_address = "http://127.0.0.1:8000/mcp"

### Interazione del client con il server

Una volta connesso al server, il client può chiedere prima di tutto l'elenco degli strumenti messi a disposizione del server, la cui documentazione è visualizzabile come una lista di schemi JSON.  
Questi strumenti sono funzioni Python, implementate in `xlserver.py` e decorate con `@mcp.tool()`.

In [5]:
MCP_server_tools = await client_tools.get_tool_list(server_address)
if not MCP_server_tools["result"]:
    print(f"Errore di connessione al server: {MCP_server_tools['data']}")
tool_schemas = client_tools.get_tool_schemas(MCP_server_tools["data"])
tool_schemas

[{'type': 'function',
  'function': {'name': 'create_workbook',
   'description': 'Crea un file xlsx dal nome specificato. ',
   'parameters': {'type': 'object',
    'properties': {'filename': {'description': 'Il nome del file xlsx da creare',
      'title': 'Filename',
      'type': 'string'}},
    'required': ['filename']}}},
 {'type': 'function',
  'function': {'name': 'write_data_in_cell',
   'description': 'Assumendo che il file xlsx specificato già esista e non debba essere creato, scrive il valore o la formula specificati nella cella specificata. ',
   'parameters': {'type': 'object',
    'properties': {'filename': {'description': 'Il nome del file xlsx in cui scrivere i dati',
      'title': 'Filename',
      'type': 'string'},
     'cell': {'description': 'La cella in cui scrivere',
      'title': 'Cell',
      'type': 'string'},
     'data': {'description': 'Il valore o la formula da scrivere nella cella',
      'title': 'Data'}},
    'required': ['filename', 'cell', 'data']}

Questi schemi JSON mostrano quali richieste un client MCP può inviare al server, specificando il nome di uno strumento, dunque di una funzione Python, e gli eventuali parametri da passare. Si possono perciò inviare richieste indipendentemente dall'uso di un modello di linguaggio, come in questo esempio.

In [6]:
function_name = "create_workbook"
function_arguments = {"filename": "prova.xlsx"}
function_name2 = "write_data_in_cell"
function_arguments2 = {"filename": "prova.xlsx", "cell": "A1", "data": 1234}

client_tools.print_msgs(await client_tools.lowlevel_call_MCP_server(server_address, function_name, function_arguments))
client_tools.print_msgs(await client_tools.lowlevel_call_MCP_server(server_address, function_name2, function_arguments2))

'File prova.xlsx creato con successo.'
'Dati scritti nel file prova.xlsx correttamente.'


A questo punto, e assumendo che un modello di linguaggio sia raggiungibile,
1. lo si può interpellare, inviandogli richieste in italiano che il modello traduce in richieste in formato JSON, grazie alle capacità di _function calling_ del modello stesso (se si usa Qwen3 si può provare ad aggiungere in fondo alla richiesta lo switch `/no_think` per rendere l'elaborazione più rapida),
2. richieste che il client MCP invia al server MCP,
3. che le esegue e produce il risultato richiesto.

In [8]:
model = await client_tools.connect_to_model() # per connettersi al modello via http con i parametri di default di LM Studio

prompt = "Crea un file Excel chiamato 'prova.xlsx'. /no_think."
model_response = client_tools.do(model, prompt, tool_schemas) # passo 1
client_tools.print_tools([tool_call.function for tool_call in model_response.choices[0].message.tool_calls]) # passo 2 # type: ignore
client_tools.print_msgs(await client_tools.call_MCP_server(server_address, model_response)) # passo 3

[Function(arguments='{"filename":"prova.xlsx"}', name='create_workbook')]
'File prova.xlsx creato correttamente.'


È interessante che, nonostante le sue piccole dimensioni, il modello di linguaggio sia in grado di "comprendere" se una richiesta che ha ricevuto deve essere tradotta in una successione di più chiamate a strumenti.

In [9]:
prompt = "Crea un file Excel chiamato 'prova.xlsx' e scrivi il numero 1234 nella cella A1. /no_think."
model_response = client_tools.do(model, prompt, tool_schemas)
client_tools.print_tools([tool_call.function for tool_call in model_response.choices[0].message.tool_calls]) # type: ignore
client_tools.print_msgs(await client_tools.call_MCP_server(server_address, model_response))

[   Function(arguments='{"filename":"prova.xlsx"}', name='create_workbook'),
    Function(arguments='{"filename":"prova.xlsx","cell":"A1","data":1234}', name='write_data_in_cell')]
'File prova.xlsx creato correttamente.'
'Dati scritti nel file prova.xlsx correttamente.'


Dato questo principio, quanto complesse possono essere le richieste traducibili in successioni di chiamate a strumenti diventa una questione di quanto sofisticato è il modello di linguaggio che si sta usando.

In [10]:
prompt = """Crea il file 'prova.xlsx', nelle celle da A1 ad A5 scrivi i numeri interi da 1 a 5,
e nelle celle da B1 a B5 scrivi la formula per calcolare il quadrato del numero
nella cella nella stessa riga della colonna A. /no_think."""
model_response = client_tools.do(model, prompt, tool_schemas)
client_tools.print_tools([tool_call.function for tool_call in model_response.choices[0].message.tool_calls]) # type: ignore
client_tools.print_msgs(await client_tools.call_MCP_server(server_address, model_response))

[   Function(arguments='{"filename":"prova.xlsx"}', name='create_workbook'),
    Function(arguments='{"filename":"prova.xlsx","first_cell":"A1","last_cell":"A5","data":[1,2,3,4,5]}', name='write_data_in_range'),
    Function(arguments='{"filename":"prova.xlsx","first_cell":"B1","last_cell":"B5","data":["=A1^2","=A2^2","=A3^2","=A4^2","=A5^2"]}', name='write_data_in_range')]
'File prova.xlsx creato correttamente.'
'Dati scritti nel file prova.xlsx correttamente.'
'Dati scritti nel file prova.xlsx correttamente.'


In [12]:
prompt = """Crea il file 'prova.xlsx' e nelle celle da A1 a C5
scrivi la formula per generare numeri casuali decimali tra 10 e 20. /no_think"""
model_response = client_tools.do(model, prompt, tool_schemas)
client_tools.print_tools([tool_call.function for tool_call in model_response.choices[0].message.tool_calls]) # type: ignore
client_tools.print_msgs(await client_tools.call_MCP_server(server_address, model_response))

[   Function(arguments='{"filename":"prova.xlsx"}', name='create_workbook'),
    Function(arguments='{"filename":"prova.xlsx","first_cell":"A1","last_cell":"C5","data":[["=RANDBETWEEN(10,20)"],["=RANDBETWEEN(10,20)"],["=RANDBETWEEN(10,20)"],["=RANDBETWEEN(10,20)"],["=RANDBETWEEN(10,20)"]]}', name='write_data_in_range')]
'File prova.xlsx creato correttamente.'
'Dati scritti nel file prova.xlsx correttamente.'
